In [1]:
import numpy as np
import pandas as pd 
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import torch
import transformers
torch.__version__, transformers.__version__

('2.0.0', '4.28.1')

In [3]:
torch.cuda.is_available()

True

In [4]:
data=pd.read_csv('../input/circa-dataset/circa-data.tsv',sep='\t')
data.head()

,id,context,question-X,canquestion-X,answer-Y,judgements,goldstandard1,goldstandard2
0,0,Y has just travelled from a different city to ...,Are you employed?,I am employed .,I'm a veterinary technician.,Yes#Yes#Yes#Yes#Yes,Yes,Yes
1,1,X wants to know about Y's food preferences.,Are you a fan of Korean food?,I am a fan of Korean food .,I wouldn't say so,Probably no#No#No#No#Probably yes / sometimes yes,No,No
2,2,Y has just told X that he/she is thinking of b...,Are you bringing any pets into the flat?,I am bringing pets into the flat .,I do not own any pets,No#No#No#No#No,No,No
3,3,X wants to know what activities Y likes to do ...,Would you like to get some fresh air in your f...,I would like to get fresh air in my free time .,I am desperate to get out of the city.,"Yes#Yes, subject to some conditions#Probably y...",Yes,Yes
4,4,X and Y are childhood neighbours who unexpecte...,Is your family still living in the neighborhood?,My family is living in the neighborhood .,My parents are snowbirds now.,"No#In the middle, neither yes nor no#Probably ...","In the middle, neither yes nor no","In the middle, neither yes nor no"


In [5]:
data = data.dropna()
data['goldstandard2'].value_counts()

Yes                                  15745
No                                   11985
Yes, subject to some conditions       2580
In the middle, neither yes nor no      701
Other                                  504
Name: goldstandard2, dtype: int64

In [6]:
data['label_2'] = data['goldstandard2'].copy()

data['labels'] = data['label_2'].map({'Yes': 0, 'No': 1, 'Yes, subject to some conditions': 2,
                                  'In the middle, neither yes nor no': 3, 'Other': 4})

In [7]:
data.reset_index(drop=True, inplace=True)

In [8]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

train , test = train_test_split(data, test_size = 0.20)

# train['sep_token'] = '[SEP]'
# train['cls_token'] = '[CLS]'
# train['text'] = train['cls_token'] + \
#                     train['context'] + train['sep_token']+ train['question-X'] + \
#                     train['sep_token'] + train['answer-Y'] + \
#                 train['sep_token']


X = train[['question-X', 'answer-Y']]
y = tf.keras.utils.to_categorical(train.labels, num_classes=5)


ref_train = train[['question-X', 'answer-Y', 'labels']]

df_train, df_val = train_test_split(ref_train, stratify=ref_train['labels'], random_state=42, test_size = 0.15)
print(train.shape, test.shape)
print(df_train.shape)
print(df_val.shape)

(25212, 10) (6303, 10)
(21430, 3)
(3782, 3)


In [9]:
from transformers import AutoTokenizer, GPTNeoForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
model = GPTNeoForSequenceClassification.from_pretrained("EleutherAI/gpt-neo-125M", num_labels=5)

Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at EleutherAI/gpt-neo-125M and are newly initialized: ['transformer.h.11.attn.attention.bias', 'transformer.h.5.attn.attention.bias', 'score.weight', 'transformer.h.3.attn.attention.bias', 'transformer.h.9.attn.attention.bias', 'transformer.h.7.attn.attention.bias', 'transformer.h.1.attn.attention.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [113]:
tokenizer.pad_token = tokenizer.eos_token

In [28]:
test

,id,context,question-X,canquestion-X,answer-Y,judgements,goldstandard1,goldstandard2,label_2,labels
15273,16776,X wants to know what sorts of books Y likes to...,Do you like books with movie remakes?,I like books with movie remakes .,"I enjoy reading the book, then watching the mo...",Yes#Yes#Yes#Yes#Yes,Yes,Yes,Yes,0
3102,3430,Y has just moved into a neighbourhood and meet...,Do you watch documentaries very often?,I watch documentaries very often .,I prefer thrillers.,"No#No#No#In the middle, neither yes nor no#No",No,No,No,1
23107,25163,X wants to know what activities Y likes to do ...,Do you like to spend your weekends at home?,I like to spend my weekends at home .,I try to avoid leaving my house as much as pos...,No#Yes#Yes#Yes#Yes,Yes,Yes,Yes,0
25753,28055,X wants to know about Y's music preferences.,Would you like to go to the symphony?,I would like to go to the symphony .,It is something I would love to see.,Yes#Yes#Yes#Yes#Yes,Yes,Yes,Yes,0
8837,9766,X wants to know about Y's music preferences.,Do you like Michael Buble?,I like Michael Buble .,I'm into that big band sound.,Yes#No#Yes#No#Yes,Yes,Yes,Yes,0
...,...,...,...,...,...,...,...,...,...,...
17100,18746,Y has just moved into a neighbourhood and meet...,Do you enjoy sports?,I enjoy sports .,I can watch any sport on TV.,"Yes, subject to some conditions#Yes#I am not s...",Yes,Yes,Yes,0
177,202,Y has just moved into a neighbourhood and meet...,Have you lived in this area long?,I have lived in this area long .,Several of my neighbors remember me in diapers.,Yes#Yes#Yes#Yes#Yes,Yes,Yes,Yes,0
29342,31878,Y has just told X that he/she is considering s...,Have you worked with sql previously?,I have worked with sql previously .,I've only worked with HTML.,No#No#No#No#No,No,No,No,1
29691,32267,Y has just travelled from a different city to ...,Would you like to take the spare key to my house?,I would like to take the spare key to my house .,That would make things a lot easier.,Yes#Yes#Yes#Yes#Yes,Yes,Yes,Yes,0


In [11]:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(test)

tokenize_function = lambda e: tokenizer(e['question-X'], e['answer-Y'], truncation=True, padding='max_length', max_length=512)

dataset_train = dataset_train.map(tokenize_function, batched=True)
dataset_val = dataset_val.map(tokenize_function, batched=True)
dataset_test = dataset_test.map(tokenize_function, batched=True)

  0%|          | 0/22 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [12]:
dataset_train.set_format(type='torch')
dataset_val.set_format(type='torch')
dataset_test.set_format(type='torch')

In [13]:
dataset_train

Dataset({
    features: ['question-X', 'answer-Y', 'labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 21430
})

In [14]:
# dataset_train.save_to_disk('train_gpt_neo_125')
# dataset_val.save_to_disk('val_gpt_neo_125')
# dataset_test.save_to_disk('test_gpt_neo_125')

In [15]:
# from datasets import load_dataset, load_from_disk
# dataset_train = load_from_disk('/kaggle/input/gpt-neo-125/train_gpt_neo_125')
# dataset_val = load_from_disk('/kaggle/input/gpt-neo-125/val_gpt_neo_125')
# dataset_test = load_from_disk('/kaggle/input/gpt-neo-125/test_gpt_neo_125')

In [112]:
model.config.pad_token_id = model.config.eos_token_id

In [17]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    #tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()
    return {'accuracy' : accuracy_score(labels, predictions)}




args = TrainingArguments(
        output_dir = 'temp/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=3e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
)

trainer = Trainer(
        model=model,                         
        args=args,                  
        train_dataset=dataset_train,         
        eval_dataset=dataset_val,            
        compute_metrics=compute_metrics
)

trainer.train()   

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,0.494200,0.466130,0.829984
2,0.294200,0.477603,0.845849
3,0.132200,0.619853,0.849551


TrainOutput(global_step=4020, training_loss=0.3299283002739522, metrics={'train_runtime': 3852.8072, 'train_samples_per_second': 16.687, 'train_steps_per_second': 1.043, 'total_flos': 1.679376268394496e+16, 'train_loss': 0.3299283002739522, 'epoch': 3.0})

In [18]:
model.eval()
trainer.predict(dataset_test).metrics

{'test_loss': 0.5947389006614685,
 'test_accuracy': 0.8537204505790893,
 'test_runtime': 121.9171,
 'test_samples_per_second': 51.699,
 'test_steps_per_second': 3.232}

In [25]:
trainer.save_model("my_trained_model")

In [81]:
data_template = test.copy()
data_template = data_template.iloc[0:0]
data_template

,id,context,question-X,canquestion-X,answer-Y,judgements,goldstandard1,goldstandard2,label_2,labels


In [82]:
questions = ["Did you eat the last piece of cake?", 
            "Are you coming to the meeting tomorrow?",
            "Did you finish the report I asked you to complete?",
            "Have you decided on your vacation destination?",
            "Did you enjoy the service that we provided?",
            "Did you like the book you just finished?",
            "Are you willing to contribute to the charity?",
            "Want to go to the waterpark this weekend?",
            "Will you support the new policy?"]
answers = ["I really enjoyed it. It was delicious!",
          "I'll need to check my schedule and see what's going on.",
          "I worked on it for a while, but I got busy with other things.",
          "I'm thinking about a few places, but I need to check my budget first.",
          "I would have to say yes to disappointment I'm afraid.",
          "The ending was unexpected, but I had mixed feelings about the characters.",
          "I'm open to it, provided it's a cause I believe in.",
          "Let's get soaked!",
          "Haven't made up my mind yet."]

In [124]:
for x in range(0,9):
    data_template.loc[x, 'question-X'] = questions[x]
    data_template.loc[x, 'answer-Y'] = answers[x]
    data_template.loc[x, 'labels'] = "random"

    
data_template['labels'] = data_template['labels'].map({'random': 0})
data_template

,id,context,question-X,canquestion-X,answer-Y,judgements,goldstandard1,goldstandard2,label_2,labels
0,NaN,NaN,Did you eat the last piece of cake?,NaN,I really enjoyed it. It was delicious!,NaN,NaN,NaN,NaN,0
1,NaN,NaN,Are you coming to the meeting tomorrow?,NaN,I'll need to check my schedule and see what's ...,NaN,NaN,NaN,NaN,0
2,NaN,NaN,Did you finish the report I asked you to compl...,NaN,"I worked on it for a while, but I got busy wit...",NaN,NaN,NaN,NaN,0
3,NaN,NaN,Have you decided on your vacation destination?,NaN,"I'm thinking about a few places, but I need to...",NaN,NaN,NaN,NaN,0
4,NaN,NaN,Did you enjoy the service that we provided?,NaN,I would have to say yes to disappointment I'm ...,NaN,NaN,NaN,NaN,0
5,NaN,NaN,Did you like the book you just finished?,NaN,"The ending was unexpected, but I had mixed fee...",NaN,NaN,NaN,NaN,0
6,NaN,NaN,Are you willing to contribute to the charity?,NaN,"I'm open to it, provided it's a cause I believ...",NaN,NaN,NaN,NaN,0
7,NaN,NaN,Want to go to the waterpark this weekend?,NaN,Let's get soaked!,NaN,NaN,NaN,NaN,0
8,NaN,NaN,Will you support the new policy?,NaN,Haven't made up my mind yet.,NaN,NaN,NaN,NaN,0


In [108]:
# data_template

In [125]:
data_samples = Dataset.from_pandas(data_template)
tokenize_function = lambda e: tokenizer(e['question-X'], e['answer-Y'], truncation=True, padding='max_length', max_length=512)
data_samples = data_samples.map(tokenize_function, batched=True)
data_samples.set_format(type='torch')

  0%|          | 0/1 [00:00<?, ?ba/s]

In [126]:
# samples_predictions = trainer.predict(data_samples)

In [127]:
samples_predictions

PredictionOutput(predictions=array([[10.126816  , -0.42643693, -1.6540213 ,  1.2501225 , -7.800314  ],
       [ 1.9932374 ,  5.318508  , -1.2673482 ,  4.537936  , -9.464727  ],
       [ 3.4238472 ,  7.608694  , -3.7428658 , -0.32347912, -4.197444  ],
       [ 1.7642367 ,  5.802375  ,  1.6382964 ,  1.1260295 , -4.7271757 ],
       [ 2.2850184 ,  8.788585  , -0.3716575 ,  0.9038954 , -7.771817  ],
       [ 0.71667373,  4.247594  , -2.1853766 ,  2.7174733 , -1.9434562 ],
       [ 6.388684  ,  1.2695456 ,  1.0262202 ,  1.6515431 , -8.621865  ],
       [ 7.5551505 ,  0.30653703,  2.2056208 , -1.8545654 , -5.0860844 ],
       [ 3.3414044 ,  5.409586  , -0.3273251 ,  3.1183643 , -5.0964146 ]],
      dtype=float32), label_ids=array([0, 0, 0, 0, 0, 0, 0, 0, 0]), metrics={'test_loss': 2.72975492477417, 'test_accuracy': 0.3333333333333333, 'test_runtime': 0.1883, 'test_samples_per_second': 47.798, 'test_steps_per_second': 5.311})

In [128]:
import numpy as np
preds = np.argmax(samples_predictions.predictions, axis=1)
preds

array([0, 1, 1, 1, 1, 1, 0, 0, 1])